In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn import linear_model, metrics
from sklearn.linear_model import Lasso

In [6]:
data = pd.read_csv('test_data_csv.csv')
print(data.head())
print(data.shape)

   Unnamed: 0    MSN StateCode  Year   Amount   State  CO2 Emissions (Mmt)  \
0        7595  BDFDB        AK  2020      0.0  Alaska            35.967527   
1        7596  BDPRP        AK  2020      0.0  Alaska            35.967527   
2        7597  BFFDB        AK  2020      0.0  Alaska            35.967527   
3        7598  BFPRP        AK  2020      0.0  Alaska            35.967527   
4        7599  CLPRB        AK  2020  15523.0  Alaska            35.967527   

   TotalNumberofInvestments  TotalAmountofAssistance  
0                      37.0                9567164.0  
1                      37.0                9567164.0  
2                      37.0                9567164.0  
3                      37.0                9567164.0  
4                      37.0                9567164.0  
(1508, 9)


In [7]:
clean_df = data.query("StateCode != 'DC'").query("StateCode != 'US'").query("StateCode != 'X3'").query("StateCode != 'X5'")
print(clean_df.head())
print(clean_df.shape)
print(clean_df.StateCode.unique())
print(clean_df.StateCode.unique().size)

   Unnamed: 0    MSN StateCode  Year   Amount   State  CO2 Emissions (Mmt)  \
0        7595  BDFDB        AK  2020      0.0  Alaska            35.967527   
1        7596  BDPRP        AK  2020      0.0  Alaska            35.967527   
2        7597  BFFDB        AK  2020      0.0  Alaska            35.967527   
3        7598  BFPRP        AK  2020      0.0  Alaska            35.967527   
4        7599  CLPRB        AK  2020  15523.0  Alaska            35.967527   

   TotalNumberofInvestments  TotalAmountofAssistance  
0                      37.0                9567164.0  
1                      37.0                9567164.0  
2                      37.0                9567164.0  
3                      37.0                9567164.0  
4                      37.0                9567164.0  
(1450, 9)
['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DE' 'FL' 'GA' 'HI' 'IA' 'ID' 'IL'
 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC' 'ND'
 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA

In [8]:
# Convert everything to billion btu
## Convert biodiesel to billion btu
clean_df = clean_df
# Remove the BDFDB, BFFDB, BFPRP, EMFDB
clean_df = clean_df.query("MSN != 'BDFDB'").query("MSN != 'BFFDB'").query("MSN != 'BFPRP'").query("MSN != 'EMFDB'")
print(clean_df.shape)
# Convert biodiesel to energy
biodiesel_converted = clean_df.query("MSN == 'BDPRP'")
# print(biodiesel_converted['Amount'])
biodiesel_converted.loc[:, "Amount"] = biodiesel_converted.loc[:, "Amount"] * 5.46
print(biodiesel_converted)
non_bdprp = clean_df.query("MSN != 'BDPRP'")
clean_df = pd.concat([non_bdprp, biodiesel_converted])
print(clean_df.shape)

(1250, 9)
      Unnamed: 0    MSN StateCode  Year    Amount           State  \
1           7596  BDPRP        AK  2020      0.00          Alaska   
30          7625  BDPRP        AL  2020   1567.02         Alabama   
59          7654  BDPRP        AR  2020   8479.38        Arkansas   
88          7683  BDPRP        AZ  2020      0.00         Arizona   
117         7712  BDPRP        CA  2020   4892.16      California   
146         7741  BDPRP        CO  2020      0.00        Colorado   
175         7770  BDPRP        CT  2020   2642.64     Connecticut   
233         7828  BDPRP        DE  2020      0.00        Delaware   
262         7857  BDPRP        FL  2020    169.26         Florida   
291         7886  BDPRP        GA  2020   1332.24         Georgia   
320         7915  BDPRP        HI  2020    475.02          Hawaii   
349         7944  BDPRP        IA  2020  45629.22            Iowa   
378         7973  BDPRP        ID  2020      0.00           Idaho   
407         8002  BDPRP 

/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [9]:
# Convert ethanol to energy
ethanol_converted = clean_df.query("MSN == 'ENPRP'")
print(ethanol_converted['Amount'])
ethanol_converted.loc[:, "Amount"] = ethanol_converted.loc[:, "Amount"] * 3.192
print(ethanol_converted['Amount'])
non_enprp = clean_df.query("MSN != 'ENPRP'")
clean_df = pd.concat([non_enprp, ethanol_converted])
print(clean_df.shape)

9           0.0
38          0.0
67          0.0
96          0.0
125      2710.0
154      3212.0
183         0.0
241         0.0
270         0.0
299      1169.0
328         0.0
357     92137.0
386       365.0
415     33807.0
444     23148.0
473     12721.0
502       928.0
531         0.0
560         0.0
589         0.0
618         0.0
647      7381.0
676     27284.0
705      6098.0
734         0.0
763         0.0
792         0.0
821     12280.0
850     42863.0
879         0.0
908         0.0
937         0.0
966         0.0
995      1753.0
1024    12725.0
1053        0.0
1082      790.0
1111     2289.0
1140        0.0
1169        0.0
1198    27664.0
1227     4033.0
1256     5549.0
1314        0.0
1343       32.0
1372        0.0
1401        0.0
1430    10990.0
1459        0.0
1488        0.0
Name: Amount, dtype: float64
9            0.000
38           0.000
67           0.000
96           0.000
125       8650.320
154      10252.704
183          0.000
241          0.000
270          0.000


/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [10]:
clean_df = clean_df.query("MSN != 'CLPRK'").query("MSN != 'CLPRP'").query("MSN != 'COPRK'") \
                    .query("MSN != 'PAPRP'").query("MSN != 'NGMPK'").query("MSN != 'NGMPP'")
print(clean_df)

      Unnamed: 0    MSN StateCode  Year     Amount          State  \
4           7599  CLPRB        AK  2020   15523.00         Alaska   
10          7605  GETCB        AK  2020     186.00         Alaska   
11          7606  HYTCB        AK  2020   15475.00         Alaska   
12          7607  NCPRB        AK  2020   16851.00         Alaska   
13          7608  NGMPB        AK  2020  376283.00         Alaska   
...          ...    ...       ...   ...        ...            ...   
1372        8967  ENPRP        VT  2020       0.00        Vermont   
1401        8996  ENPRP        WA  2020       0.00     Washington   
1430        9025  ENPRP        WI  2020   35080.08      Wisconsin   
1459        9054  ENPRP        WV  2020       0.00  West Virginia   
1488        9083  ENPRP        WY  2020       0.00        Wyoming   

      CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
4               35.967527                      37.0                9567164.0  
10           

In [11]:
# Sort and reindex dataframe
print(clean_df.sort_values('Unnamed: 0'))
clean_df.reset_index(drop=True, inplace=True)

      Unnamed: 0    MSN StateCode  Year   Amount    State  \
1           7596  BDPRP        AK  2020      0.0   Alaska   
4           7599  CLPRB        AK  2020  15523.0   Alaska   
9           7604  ENPRP        AK  2020      0.0   Alaska   
10          7605  GETCB        AK  2020    186.0   Alaska   
11          7606  HYTCB        AK  2020  15475.0   Alaska   
...          ...    ...       ...   ...      ...      ...   
1503        9098  WDPRB        WY  2020   4757.0  Wyoming   
1504        9099  WDTCB        WY  2020   4757.0  Wyoming   
1505        9100  WSTCB        WY  2020      0.0  Wyoming   
1506        9101  WWPRB        WY  2020   4757.0  Wyoming   
1507        9102  WYTCB        WY  2020  48362.0  Wyoming   

      CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
1               35.967527                      37.0                9567164.0  
4               35.967527                      37.0                9567164.0  
9               35.967527     

In [12]:
print(clean_df)

     Unnamed: 0    MSN StateCode  Year     Amount          State  \
0          7599  CLPRB        AK  2020   15523.00         Alaska   
1          7605  GETCB        AK  2020     186.00         Alaska   
2          7606  HYTCB        AK  2020   15475.00         Alaska   
3          7607  NCPRB        AK  2020   16851.00         Alaska   
4          7608  NGMPB        AK  2020  376283.00         Alaska   
..          ...    ...       ...   ...        ...            ...   
945        8967  ENPRP        VT  2020       0.00        Vermont   
946        8996  ENPRP        WA  2020       0.00     Washington   
947        9025  ENPRP        WI  2020   35080.08      Wisconsin   
948        9054  ENPRP        WV  2020       0.00  West Virginia   
949        9083  ENPRP        WY  2020       0.00        Wyoming   

     CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
0              35.967527                      37.0                9567164.0  
1              35.967527   

In [14]:
# Test Query Code
test_query = clean_df.query("StateCode == 'AK' & Year == 2020")
print(test_query)
print(test_query.iloc[0, 8])
print(type(test_query.iloc[0, 8]))

# See how many features we have remaining
print(clean_df.MSN.unique())
print(clean_df.MSN.unique().size)

     Unnamed: 0    MSN StateCode  Year     Amount   State  \
0          7599  CLPRB        AK  2020    15523.0  Alaska   
1          7605  GETCB        AK  2020      186.0  Alaska   
2          7606  HYTCB        AK  2020    15475.0  Alaska   
3          7607  NCPRB        AK  2020    16851.0  Alaska   
4          7608  NGMPB        AK  2020   376283.0  Alaska   
5          7611  NUETB        AK  2020        0.0  Alaska   
6          7612  PAPRB        AK  2020   932482.0  Alaska   
7          7614  REPRB        AK  2020    24682.0  Alaska   
8          7615  SOTCB        AK  2020       62.0  Alaska   
9          7616  TEPRB        AK  2020  1348969.0  Alaska   
10         7617  TETCB        AK  2020   640132.0  Alaska   
11         7618  WDEXB        AK  2020        0.0  Alaska   
12         7619  WDPRB        AK  2020     7423.0  Alaska   
13         7620  WDTCB        AK  2020     7423.0  Alaska   
14         7621  WSTCB        AK  2020      407.0  Alaska   
15         7622  WWPRB  

In [15]:
# alaska_df = clean_df.loc[clean_df["StateCode"] == "AK"]
print(clean_df)
print(clean_df.StateCode.unique())
print(clean_df.StateCode.unique().size)

     Unnamed: 0    MSN StateCode  Year     Amount          State  \
0          7599  CLPRB        AK  2020   15523.00         Alaska   
1          7605  GETCB        AK  2020     186.00         Alaska   
2          7606  HYTCB        AK  2020   15475.00         Alaska   
3          7607  NCPRB        AK  2020   16851.00         Alaska   
4          7608  NGMPB        AK  2020  376283.00         Alaska   
..          ...    ...       ...   ...        ...            ...   
945        8967  ENPRP        VT  2020       0.00        Vermont   
946        8996  ENPRP        WA  2020       0.00     Washington   
947        9025  ENPRP        WI  2020   35080.08      Wisconsin   
948        9054  ENPRP        WV  2020       0.00  West Virginia   
949        9083  ENPRP        WY  2020       0.00        Wyoming   

     CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
0              35.967527                      37.0                9567164.0  
1              35.967527   

In [16]:
### Test on all states
# Generate all years and all states
all_years = [2020]
all_states = clean_df.StateCode.unique()

In [19]:
print(clean_df)
print(all_states)
print(all_states.size)

     Unnamed: 0    MSN StateCode  Year     Amount          State  \
0          7599  CLPRB        AK  2020   15523.00         Alaska   
1          7605  GETCB        AK  2020     186.00         Alaska   
2          7606  HYTCB        AK  2020   15475.00         Alaska   
3          7607  NCPRB        AK  2020   16851.00         Alaska   
4          7608  NGMPB        AK  2020  376283.00         Alaska   
..          ...    ...       ...   ...        ...            ...   
945        8967  ENPRP        VT  2020       0.00        Vermont   
946        8996  ENPRP        WA  2020       0.00     Washington   
947        9025  ENPRP        WI  2020   35080.08      Wisconsin   
948        9054  ENPRP        WV  2020       0.00  West Virginia   
949        9083  ENPRP        WY  2020       0.00        Wyoming   

     CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
0              35.967527                      37.0                9567164.0  
1              35.967527   

In [21]:
# Create Response Variable

num_datapoints = len(all_years) * len(all_states) 
num_cols = 1
y = pd.DataFrame(index=range(num_datapoints), columns=["TotalAmountofAssistance"])
investment_vals = []
for year in all_years:
    for state in all_states:
        state_query = "StateCode == " + "'" + str(state) + "'"
        year_query = "Year == " + str(year)
        investment_year_state = clean_df.query(year_query + " & " + state_query)
        # print("INVESTMENT YEAR QUERY DF")
        # print(investment_year.head())
        # print(investment_year.shape)
        investment_year_val = investment_year_state.iloc[0, 8]
        investment_vals.append(investment_year_val)
for yIdx in range(num_datapoints):
    y.at[yIdx, "TotalAmountofAssistance"] = investment_vals[yIdx]
# y.at[0]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][0]
# print(y)
# y.at[1]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][1519]
# y.at[2]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][3038]
# y.at[3]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][4557]
# y.at[4]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][6076]
# for investment_idx in range(5):
#     print(investment_idx)
#     print(data["TotalAmountofAssistance"][3038])
#     row_idx = 1519 * investment_idx
#     y.at[investment_idx]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][row_idx]
print(y)

y.to_csv('assistance_test.csv')

   TotalAmountofAssistance
0                9567164.0
1                1174323.0
2               61208016.0
3               23756565.0
4               58722209.0
5                6304206.0
6                 491729.0
7               23750266.0
8                8467648.0
9                7307409.0
10               1153848.0
11              63476066.0
12                733911.0
13               3526853.0
14               8555854.0
15               2802130.0
16              11046095.0
17               5171076.0
18              21021782.0
19               1459180.0
20               1718854.0
21               3543100.0
22              29357835.0
23               4677028.0
24               2102663.0
25                542585.0
26             176369306.0
27                686913.0
28               9134015.0
29                996776.0
30                428543.0
31              48503359.0
32                 53672.0
33              39346669.0
34               3038155.0
35                560909.0
3